In [5]:
import pandas as pd
import datasets
import sys
import os
import json
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(parent_dir)
import utils
aspects = [ 'actionability', 'grounding_specificity', 'verifiability', 'helpfulness']

# Load the dataset
data_1 = pd.read_excel('../chatgpt/outputs/main_data_batch_1_results.xlsx', sheet_name=None)
data_2 = pd.read_excel('../chatgpt/outputs/main_data_batch_2_results.xlsx', sheet_name=None)

data =  {}

### combine the two batches
for key in data_1.keys():
    if key in data_2.keys():
        data[key] = pd.concat([data_1[key], data_2[key]])
        print(f"length of {key}: {len(data[key])}")

## remove the verifiability_extraction sheet
data.pop('verifiability_extraction')

## Remove entries that has NA in the score or rationale columns
for key in data.keys():
    score = f'chatgpt_{key}_rationale'
    ratioanele = f'chatgpt_{key}_rationale'
    data[key] = data[key].dropna(subset=[score, ratioanele])
    

length of verifiability_extraction: 9990
length of helpfulness: 9603
length of actionability: 9995
length of grounding_specificity: 9981
length of verifiability: 9937


In [6]:
for key in data.keys():
    print(key)
    print(len(data[key]))

helpfulness
9603
actionability
9995
grounding_specificity
9981
verifiability
9937


In [7]:
### convert the df to dataset and store for each aspect
for aspect in data.keys():
    dataset = datasets.Dataset.from_pandas(data[aspect]).train_test_split(test_size=0.1)
    ## add new column for the prompt do in sepratately for th train and test
    
    dataset['train'] = dataset['train'].map(lambda x: {'prompt': utils.get_prompt(x, aspect, task='train',evaluation_type='score_only')})
    dataset['test'] = dataset['test'].map(lambda x: {'prompt': utils.get_prompt(x, aspect, task='test',evaluation_type='score_only')})
    dataset.push_to_hub(f'boda/review_evaluation_automatic_labels',config_name=aspect )
    




Map:   0%|          | 0/8642 [00:00<?, ? examples/s]

Map:   0%|          | 0/961 [00:00<?, ? examples/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/9 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

...

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/9 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/5.44k [00:00<?, ?B/s]

In [8]:

merged_df = []

### aggregate the different aspects in one dataset
for i,row in data['actionability'].iterrows():
    ## remove the prompt column
    row.pop('prompt')
    failed = False
    for aspect in data.keys():
        if aspect == 'actionability':
            continue
        ### get the same row from the other aspects
        ## make sure the row exist
        if row['id'] not in data[aspect]['id'].values:
            print(f'row {row["id"]} not found in {aspect}')
            ## ignore this row
            failed = True
            continue

        aspect_row = data[aspect][data[aspect]['id'] == row['id']].iloc[0]
        row[f'chatgpt_{aspect}_score'] = aspect_row[f'chatgpt_{aspect}_score']
        row[f'chatgpt_{aspect}_rationale'] = aspect_row[f'chatgpt_{aspect}_rationale']
        ## if the score or rationale is none then ignore this row
        if pd.isna(aspect_row[f'chatgpt_{aspect}_score']):
            print(f'row {row["id"]} has no score in {aspect}')
            failed = True
            continue
        # row[f'{aspect}_prompt'] = aspect_row['prompt']
    if not failed:
        merged_df.append(row)
    
merged_df = pd.DataFrame(merged_df)


## convert the df to dataset and push to hub
dataset = datasets.Dataset.from_pandas(merged_df).train_test_split(test_size=0.1)
## add new column for the prompt
dataset['train'] = dataset['train'].map(lambda x: {'prompt': utils.get_prompt(x, 'all', task='train',evaluation_type='score_only')})
dataset['test'] = dataset['test'].map(lambda x: {'prompt': utils.get_prompt(x, 'all', task='test',evaluation_type='score_only')})
dataset.push_to_hub(f'boda/review_evaluation_automatic_labels',config_name='all' )

row 6848 not found in verifiability
row 8106 not found in verifiability
row 10984 not found in verifiability
row 1030 not found in verifiability
row 66 not found in verifiability
row 5877 not found in verifiability
row 11472 not found in helpfulness
row 942 not found in verifiability
row 9808 not found in verifiability
row 2387 not found in verifiability
row 1367 not found in verifiability
row 351 not found in verifiability
row 2375 not found in verifiability
row 3628 not found in verifiability
row 11741 not found in verifiability
row 2066 not found in verifiability
row 5307 not found in verifiability
row 1860 not found in verifiability
row 10102 not found in verifiability
row 5252 not found in verifiability
row 4055 not found in verifiability
row 4704 not found in verifiability
row 11737 not found in verifiability
row 971 not found in verifiability
row 7217 not found in verifiability
row 9030 not found in verifiability
row 3834 not found in verifiability
row 1197 not found in helpfuln

Map:   0%|          | 0/8581 [00:00<?, ? examples/s]

Map:   0%|          | 0/954 [00:00<?, ? examples/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/9 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/5.49k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/boda/review_evaluation_automatic_labels/commit/6bb2f3dc2f8d31ac5876be341187a6dea8ccf457', commit_message='Upload dataset', commit_description='', oid='6bb2f3dc2f8d31ac5876be341187a6dea8ccf457', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/boda/review_evaluation_automatic_labels', endpoint='https://huggingface.co', repo_type='dataset', repo_id='boda/review_evaluation_automatic_labels'), pr_revision=None, pr_num=None)

In [9]:
for aspect in aspects:
    print(merged_df[f'chatgpt_{aspect}_score'].unique())

### get the rows that has the nan values
nan_rows = merged_df[merged_df.isna().any(axis=1)]


[1 5 4 2 3]
[2 3 5 1 4]
['3' '4' 'X' '1' '5' '2']
[1 5 4 3 2]


In [10]:
nan_rows

,paper_id,venue,focused_review,review_point,id,chatgpt_actionability_score,chatgpt_actionability_rationale,chatgpt_helpfulness_score,chatgpt_helpfulness_rationale,chatgpt_grounding_specificity_score,chatgpt_grounding_specificity_rationale,chatgpt_verifiability_score,chatgpt_verifiability_rationale
128,ACL_2017_371_review,ACL_2017,NaN,"- Also, regarding section 3.3, please cite app...",209,5,The review point explicitly instructs the auth...,4,The review comment provides specific and actio...,5,The comment is fully grounded as it explicitly...,X,The comment requests the citation of appropria...
183,NIPS_2018_45,NIPS_2018,NaN,2) Why are there no quantitative results for t...,6212,4,The review comment questions the absence of qu...,3,The review comment raises a valid concern abou...,4,The comment is fully grounded as it explicitly...,3,The review point questions the absence of quan...
763,ACL_2017_193_review,ACL_2017,NaN,- While it is very complete to provide the pre...,131,4,"The review point suggests two actions: first, ...",4,The review comment provides actionable feedbac...,3,The comment suggests including an informal des...,3,The review point suggests that the paper would...
839,NIPS_2018_553,NIPS_2018,NaN,2) It is not clear if the method would perform...,6501,2,The comment raises a concern about the general...,3,The review point raises a concern about the ge...,2,The comment raises a concern about the general...,X,The review point questions the generalizabilit...
1016,NIPS_2018_775,NIPS_2018,NaN,2) The novelty is mostly incremental. === Conc...,6254,1,The review comment states that the novelty of ...,3,The review point presents a mixed message. On ...,1,The comment mentions concerns about the novelt...,2,"The review point claims that ""the novelty is m..."
1131,ICLR_2022_2956,ICLR_2022,NaN,2) The authors aim to address the overfitting ...,2827,4,The review comment questions the authors' deci...,4,The review comment raises a pertinent question...,3,"The comment is specific in its critique, quest...",X,The review point consists of questions and obs...
1564,ICLR_2021_342,ICLR_2021,NaN,2. “Closely follow the rendering process in gr...,2240,1,The review comment critiques the claim that th...,3,The review comment challenges a specific claim...,5,The comment addresses a specific claim in the ...,4,The review point challenges the claim that the...
1587,ICLR_2021_342,ICLR_2021,NaN,3. “First deep generative model for unsupervis...,2159,2,The review point challenges the claim of being...,3,The review comment challenges the originality ...,3,"The comment critiques the claim of being the ""...",3,The review point challenges the claim of being...
1864,ICLR_2022_2956,ICLR_2022,NaN,3) what if you train the model long enough? Do...,2899,2,The comment raises a question about the potent...,3,The comment raises a pertinent question about ...,3,The comment raises a specific question about t...,X,The comment poses questions about the potentia...
2787,NIPS_2018_45,NIPS_2018,NaN,2) Presentation. Although the approach is clea...,6120,3,The review point highlights a missing discussi...,3,The review comment acknowledges that the appro...,3,The comment addresses the presentation of the ...,3,The review point raises a claim that the paper...


In [11]:
# columns_to_keep = ['review_point','paper_id','venue','focused_review']
# all_datasets = {}
# for aspect in aspects:
#     aspect_dataset = []

#     aspect_data = all_data[all_data[f'{aspect}_label_type'] == 'gold']
#     aspcets_columns_to_keep = columns_to_keep + [f'{aspect}_label']
#     aspect_data = aspect_data[aspcets_columns_to_keep]

#     for i,row in aspect_data.iterrows():
#         row['prompt'] = utils.get_prompt(row, aspect, task='train',evaluation_type='score_only')
#         aspect_dataset.append(row)
#     aspect_dataset = pd.DataFrame(aspect_dataset)
#     aspect_dataset = datasets.Dataset.from_pandas(aspect_dataset)
#     if '__index_level_0__' in aspect_dataset.column_names:
#         aspect_dataset = aspect_dataset.remove_columns('__index_level_0__')

#     ## split the dataset to train and validation
#     aspect_dataset = aspect_dataset.train_test_split(test_size=0.1)
#     all_datasets[aspect] = aspect_dataset

    


In [12]:
print(dataset['test']['prompt'][0])

[{'content': 'You are an expert in evaluating peer review comments with respect to different aspects. These aspects are aimed to maximize the utilization of the review comments for the authors. The primary purpose of the review is to help/guide authors in improving their drafts. Keep this in mind while evaluating the review point. Whenever you encounter a borderline case, think: “Will this review point help authors improve their draft?”. There is no correlation between the aspect score and the length of the review point.Aspect: actionability\n\n**Actionability**\n\n**Definition:** Measures the level of actionability in a review point. We evaluate actionability based on two criteria:\n\n1. **Explicit vs. Implicit**:\n   - **Explicit:** Actions or suggestions that are direct or apparent. Authors can directly identify modifications they should apply to their draft. Clarification questions should be treated as explicit statements if they give a direct action.\n   - **Implicit:** Actions th